In [61]:
import pandas as pd
import numpy as np

In [ ]:
import datetime
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.io import show
from bokeh.layouts import gridplot

In [73]:
path = 'data/seven_day_county.csv'
DF = pd.read_csv(path)
DF['date'] = pd.to_datetime(DF['date'])
DF.head()
DF[(DF['state']=='Washington') & (DF['county'] == 'Snohomish')]


,state,county,date,new_deaths,new_cases,seven_rolling_deaths,seven_rolling_cases,population,rucc_2013
0,Washington,Snohomish,2020-01-22,0,0,0.000,0.00,822083,1
1,Washington,Snohomish,2020-01-23,0,0,0.000,0.00,822083,1
2,Washington,Snohomish,2020-01-24,0,0,0.000,0.00,822083,1
4,Washington,Snohomish,2020-01-25,0,0,0.000,0.00,822083,1
7,Washington,Snohomish,2020-01-26,0,0,0.000,0.00,822083,1
...,...,...,...,...,...,...,...,...,...
171319,Washington,Snohomish,2020-05-25,1,9,0.125,2.00,822083,1
174256,Washington,Snohomish,2020-05-26,8,48,1.125,6.75,822083,1
177195,Washington,Snohomish,2020-05-27,2,21,0.375,4.75,822083,1
180138,Washington,Snohomish,2020-05-28,2,5,0.500,2.75,822083,1


In [ ]:
output_notebook()

In [ ]:
df = DF[(DF['state'] == 'Washington') & (DF['county'] == 'King')]
by_pop = df['new_deaths']/df['population'] * 1e6
df['by_pop'] = by_pop
df.head()
dates = df['date']
y = df['by_pop'].rolling(7).mean()
#p = figure(x_axis_type = 'datetime')
#p.line(x = df['date'], y = y )
#show(p)


In [152]:
def pre_graphs(state, DF, sort_by = 'adjusted'):
    counties = list(set(DF[(DF['state'] == state) ]['county'].tolist()))
    min_date = datetime.datetime(2100, 1, 1)
    max_date = datetime.datetime(1900, 1, 1)
    max_y = 0
    ranks = []
    for i in counties:
        df_c = DF[(DF['state'] == state) & (DF['county'] == i)]
        by_pop = df_c['new_deaths']/df_c['population'] * 1e6
        pop = df_c['new_deaths']
        m = round(by_pop.mean(),1)
        l = [x for x in by_pop if x != 0 ]
        r = round(len(l)/len(by_pop),2)
        if m == 0 or r <= .1:
            continue
        y = by_pop.rolling(7).mean()
        dates = df_c['date']
        if min(dates)< min_date:
            min_date = min(dates)
        if max(dates) > max_date:
            max_date = max(dates)
        if max([x for x in y.tolist() if not np.isnan(x)]) > max_y:
            max_y =  max([x for x in y.tolist() if not np.isnan(x)])
        if sort_by == 'adjusted':
            ranks.append((i, sum([x for x in y.tolist() if not np.isnan(x)])))
        else:
            ranks.append((i, sum([x for x in pop.tolist() if not np.isnan(x)])))
    return ranks, min_date, max_date, max_y
           

In [153]:
def state_counties(state):
    ps = []
    ranks, min_date, max_date, max_y = pre_graphs(state, DF)
    ranks = sorted(ranks, key = lambda x: x[1], reverse = True)
    for i in [x[0] for x in ranks]:
        df_c = DF[(DF['state'] == state) & (DF['county'] == i)]
        by_pop = df_c['new_deaths']/df_c['population'] * 1e6
        y = by_pop.rolling(7).mean()
        dates = df_c['date']
        p = figure(x_axis_type = 'datetime', plot_width = 250, plot_height = 250,
                  title = '{i}'.format(i =i, m= m, r = r), y_range = (0,max_y),
                  x_range = (min_date, max_date))
        p.vbar(x=dates, top=y) 
        p.yaxis.axis_label = 'deaths/million'
        ps.append(p)
    grid = gridplot(ps, ncols = 4)
    return grid
grid = state_counties('Washington')
show(grid)
